In [80]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge

In [81]:
def generate_observational(n_obs, beta, gamma, sigma_e, sigma_e0, seed = 42):
    np.random.seed(seed)
    e = np.random.normal(0, sigma_e, n_obs)
    e0 = np.random.normal(0, sigma_e0, n_obs)
    X = gamma * e + e0
    Y = beta * X + e    
    
    return X.reshape(-1, 1), Y


In [82]:
def generate_interventional(n_int, beta, sigma_e, seed = None):
    if seed is not None:
        np.random.seed(seed)
    X_int = np.random.normal(0, 1, n_int)
    e = np.random.normal(0, sigma_e, n_int)
    Y_int = beta * X_int + e
    return X_int.reshape(-1, 1), Y_int
    

In [83]:
def OLS_estimate(X, Y):
    model = LinearRegression(fit_intercept = False)
    model.fit(X, Y)
    return model.coef_[0]

In [84]:
def ridge_estimate(X, Y, alpha):
    model = Ridge(alpha = alpha, fit_intercept = False)
    model.fit(X, Y)
    return model.coef_[0]


# Observational Data

In [85]:
beta_true = 1.0
gamma = 0.1
sigma_e = 10.0
sigma_e0 = 1.0
n_obs = int(1e6)

In [86]:
X_obs, Y_obs = generate_observational(n_obs, beta_true, gamma, sigma_e, sigma_e0, seed = 42)
beta_hat_obs = OLS_estimate(X_obs, Y_obs)
print(beta_hat_obs)

5.99445021468357


In [87]:
n_bootstraps = 1000
beta_bootstrap = []

for i in range(n_bootstraps):
    indices = np.random.choice(n_obs, n_obs, replace=True)
    X_boot = X_obs[indices]
    Y_boot = Y_obs[indices]
    beta_boot = OLS_estimate(X_boot, Y_boot)
    beta_bootstrap.append(beta_boot)

beta_bootstrap = np.array(beta_bootstrap)
std_beta_ols = np.std(beta_bootstrap)
print(std_beta_ols)

0.004843578295397626


# Interventional Data

In [88]:
intervention_sizes = [100, 1000, 10000]
n_repeats = 100

In [89]:
for n_int in intervention_sizes:
    beta_hats =[]
    for trial in range(n_repeats):
        X_int, Y_int = generate_interventional(n_int, beta_true, sigma_e)
        b_estimate = OLS_estimate(X_int, Y_int)
        beta_hats.append(b_estimate)
    
    beta_hats = np.array(beta_hats)
    mean_beta = np.mean(beta_hats)
    std_beta = np.std(beta_hats)
    print(f"  n_int = {n_int:5d}: mean(OLS) = {mean_beta:.4f}, std = {std_beta:.4f} "
              f"(true=1.0).")
print()

  n_int =   100: mean(OLS) = 0.9629, std = 0.9202 (true=1.0).
  n_int =  1000: mean(OLS) = 0.9918, std = 0.3421 (true=1.0).
  n_int = 10000: mean(OLS) = 0.9959, std = 0.1070 (true=1.0).



# Causal Regularization

In [94]:
n_int_demo = 1000
X_int_demo, Y_int_demo = generate_interventional(n_int_demo, beta_true, sigma_e)
alpha_grid = np.arange(1000, 5000000, 100)
best_alpha = None
best_mse = float("inf")

In [95]:
for alpha in alpha_grid:
    beta_ridge = ridge_estimate(X_obs, Y_obs, alpha)
    Y_hat_int = beta_ridge * X_int_demo.ravel()
    mse = np.mean((Y_int_demo - Y_hat_int)**2)

    if mse < best_mse:
        best_mse = mse
        best_alpha = alpha

In [97]:
beta_ridge_1 = ridge_estimate(X_obs, Y_obs, best_alpha)
beta_ridge_1

1.715651495867423

In [98]:
beta_ridge_bootstrap = []
for i in range(n_bootstraps):
    indices = np.random.choice(n_obs, n_obs, replace=True)
    X_boot = X_obs[indices]
    Y_boot = Y_obs[indices]
    beta_ridge_boot = ridge_estimate(X_boot, Y_boot, best_alpha)
    beta_ridge_bootstrap.append(beta_ridge_boot)

beta_ridge_bootstrap = np.array(beta_ridge_bootstrap)
std_beta_ridge = np.std(beta_ridge_bootstrap)
print(std_beta_ridge)

0.0022650684724831926
